This code portion was in the main notebook, right before the def of the folders. 

In [ ]:
q=0

if dim==2:
    print("Dimension 2")
    loss_all = torch.zeros(int(n_iter)) 
    regularization_all_data = torch.zeros(int(n_iter))
    regularization_all_xw = torch.zeros(int(n_iter))

    G3_s = model(test_all).detach().flatten()
    G3_t = teacher(test_all).detach()

    Z3_t = torch.sum(torch.exp(-G3_t)*((x_limit*2/(n_sample-1))**2)).detach() # normalization constant for the teacher model
    lZ3_t = torch.log(Z3_t).detach() # log of the normalization constant

    Z3_s = torch.sum(torch.exp(-G3_s)*((x_limit*2/(n_sample-1))**2)).detach()
    lZ3_s = torch.log(Z3_s).detach()
    KL[0] = torch.sum((G3_s - G3_t )*torch.exp(-G3_t))*((x_limit*2/(n_sample-1))**2) + lZ3_s

    G3_s = model2(test_all).detach().flatten()
    G3_t = teacher(test_all).detach()

    Z3_t = torch.sum(torch.exp(-G3_t)*((x_limit*2/(n_sample-1))**2)).detach() # normalization constant for the teacher model
    lZ3_t = torch.log(Z3_t).detach() # log of the normalization constant

    Z3_s = torch.sum(torch.exp(-G3_s)*((x_limit*2/(n_sample-1))**2)).detach()
    lZ3_s = torch.log(Z3_s).detach()
    KL2[0] = torch.sum((G3_s - G3_t )*torch.exp(-G3_t))*((x_limit*2/(n_sample-1))**2) + lZ3_s 

    add_const= torch.sum(( - G3_t )*torch.exp(-G3_t))*((x_limit*2/(n_sample-1))**2)


Following code was in the middle of the main training loop

In [ ]:
if t%1000==0:
        
        if dim==2:
          G3_s = model(test_all).detach().flatten()

          Z3_s = torch.sum(torch.exp(-G3_s)*((x_limit*2/(n_sample-1))**2)).detach()
          lZ3_s = torch.log(Z3_s).detach() 
          KL[q] = torch.sum((G3_s - G3_t )*torch.exp(-G3_t))*((x_limit*2/(n_sample-1))**2)+ lZ3_s

          G3_s = model2(test_all).detach().flatten()
          Z3_s = torch.sum(torch.exp(-G3_s)*((x_limit*2/(n_sample-1))**2)).detach()
          lZ3_s = torch.log(Z3_s).detach() 

          KL2[q] = torch.sum((G3_s - G3_t )*torch.exp(-G3_t))*((x_limit*2/(n_sample-1))**2)+ lZ3_s

        ce[q]= part_func[q]+model(data).detach().mean()  
        q=q+1

Plotting util inside the main training loop

In [ ]:
if dim==2:
            plot_observables_2d(model, 
                        model2, 
                        teacher, 
                        test_all, 
                        x1_sample, 
                        x2_sample, 
                        n_sample, 
                        xw, 
                        xw_compare, 
                        data, 
                        foldername, 
                        KL, 
                        KL2, 
                        ce, 
                        part_func)

Next snippet is not specific to 2d

In [ ]:
if teach:
  np.save(foldername+'model_mean',model.mean.cpu().numpy())
  np.save(foldername+'teacher_mean',teacher.mean.cpu().numpy())
  np.save(foldername+'model_logstd',model.log_std.cpu().numpy())
  np.save(foldername+'teacher_logstd',teacher.log_std.cpu().numpy())
  np.save(foldername+'model_mass',model.mix_logits.cpu().numpy())
  np.save(foldername+'teacher_mass',teacher.mix_logits.cpu().numpy())
  np.save(foldername+'walkers_jarz',xw.cpu().numpy())
  np.save(foldername+'walkers_nojarz',xw_compare.cpu().numpy())
  np.save(foldername+'weights',log_weight.cpu().numpy())   
  np.save(foldername+'KLJ',KL.cpu().numpy()) 
  np.save(foldername+'KLnoJ',KL2.cpu().numpy()) 
  np.save(foldername+'ceJ',ce.cpu().numpy()) 
  np.save(foldername+'partition',part_func.cpu().numpy()) 
  torch.save(model, foldername+'modelJ')
  torch.save(teacher, foldername+'teacher')
  torch.save(model2, foldername+'modelnoJ')

In [ ]:


if dim==2:
    fig5 = plt.figure(4);fig5.clf
    plt.semilogy(KL.detach().cpu().numpy()[0:q],label = "W/ Jarzynski - On Grid")
    plt.semilogy(KL2.detach().cpu().numpy()[0:q],label = "W/o Jarzynski - On Grid")
    plt.semilogy(ce.detach().cpu().numpy()[0:q]+add_const.cpu().numpy(),label = "W/ Jarzynski - Using weights")
    #         plt.semilogy(total_norm_all.cpu().numpy()[0:t],label = "Regularization W/ Jarzynski")
    plt.legend()
    plt.title("KL Divergence")
    plt.xlabel("Adam Iterations x 1e3")
    plt.ylabel("KL")
    filename = str(t) + "_KL.png"
    plt.savefig(foldername + filename, dpi=300, bbox_inches='tight', transparent=True,facecolor='w')
    plt.close()
